In [8]:
from transformers import AutoTokenizer, AutoModel
import torch
import pandas as pd
import numpy as np
from tqdm import tqdm

from sklearn.neural_network import MLPRegressor
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from transformers import pipeline

In [4]:
df = pd.read_csv("./dataset/cuda_speedup_small.csv")
feature_extraction = pipeline('feature-extraction', model='flax-community/gpt-neo-125M-code-clippy', tokenizer='flax-community/gpt-neo-125M-code-clippy')

Some weights of GPTNeoModel were not initialized from the model checkpoint at flax-community/gpt-neo-125M-code-clippy and are newly initialized: ['h.3.attn.attention.bias', 'h.11.attn.attention.bias', 'h.7.attn.attention.bias', 'h.9.attn.attention.bias', 'h.1.attn.attention.bias', 'h.5.attn.attention.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
clean_df = df[df.cpu_exec != 0]
clean_df = clean_df[clean_df.errors == 0]
clean_df["speedup"] = clean_df.cpu_exec/clean_df.gpu_exec

In [7]:
X = []
Y = np.asarray(clean_df.speedup)
for i, row in tqdm(clean_df.iterrows()):
    cpu_features = np.asarray(feature_extraction( "[CLS] " + row.cpu_src.strip().replace("\n", "") + " [SEP]"))[0, -1, :]
    gpu_features = np.asarray(feature_extraction( "[CLS] " + row.gpu_src.strip().replace("\n", "") + " [SEP]"))[0, -1, :]
    other_features = np.asarray([row.n, row.b1, row.b2, row.b3, row.g1, row.g2, row.g3])
    features = np.concatenate((gpu_features, cpu_features, other_features), axis=None)

    X.append(features)

1079it [05:29,  3.27it/s]


In [9]:
train_x, test_x, train_y, test_y = train_test_split(X, np.array(Y))
regr = MLPRegressor(hidden_layer_sizes=(6,8,7), random_state=1, max_iter=700).fit(train_x, train_y)

In [10]:
print( "training score=", regr.score(train_x, train_y))
print( "test score=", regr.score(test_x, test_y))


training score= 0.9902819068554806
test score= 0.8103669399934312


In [11]:
regr.predict(test_x[15:30]) - test_y[15:30]

array([ 6.15156802e-01, -3.11405908e+00,  3.10232900e+00, -1.82617544e-01,
       -1.03363300e-01,  5.65327800e-01, -2.37176800e+00,  2.08479779e+00,
       -1.21823930e+02,  1.02763297e+00,  9.35883613e-01,  1.60711051e+00,
       -3.41650001e+00,  1.43166600e+00, -1.03622810e+02])

In [18]:
np_x = np.stack(X, axis=0)

In [29]:
train_small_x, test_small_x, train_small_y, test_small_y = train_test_split(np_x[:, :-6], np.array(Y))
regr = MLPRegressor(hidden_layer_sizes=(6,8,7), random_state=1, max_iter=700).fit(train_small_x, train_small_y)

In [30]:
print( "training score=", regr.score(train_small_x, train_small_y))
print( "test score=", regr.score(test_small_x, test_small_y))

training score= 0.9857577401009734
test score= 0.8885604028862447


In [33]:
# clean_df = df[df.cpu_exec != 0]
clean_w_0_df = df[df.errors == 0]
clean_w_0_df["speedup"] = clean_w_0_df.cpu_exec/clean_w_0_df.gpu_exec
X = []
Y = np.asarray(clean_w_0_df.speedup)
for i, row in tqdm(clean_w_0_df.iterrows()):
    cpu_features = np.asarray(feature_extraction( "[CLS] " + row.cpu_src.strip().replace("\n", "") + " [SEP]"))[0, -1, :]
    gpu_features = np.asarray(feature_extraction( "[CLS] " + row.gpu_src.strip().replace("\n", "") + " [SEP]"))[0, -1, :]
    other_features = np.asarray([row.n, row.b1, row.b2, row.b3, row.g1, row.g2, row.g3])
    features = np.concatenate((gpu_features, cpu_features, other_features), axis=None)

    X.append(features)
train_x, test_x, train_y, test_y = train_test_split(X, np.array(Y))
regr = MLPRegressor(hidden_layer_sizes=(6,8,7), random_state=1, max_iter=700).fit(train_x, train_y)
print( "training score=", regr.score(train_x, train_y))
print( "test score=", regr.score(test_x, test_y))

/var/folders/05/yq4yn_xn2pg9zc0str_rffrc0000gn/T/ipykernel_85765/3346575852.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_w_0_df["speedup"] = clean_w_0_df.cpu_exec/clean_w_0_df.gpu_exec
4382it [22:33,  3.24it/s]


training score= 0.9941305273654922
test score= 0.9832769245553491


In [36]:
regr.predict(test_x[30:50]), test_y[30:50]

(array([ 1.11200033,  0.81234882, -0.69449605, -0.42519125, -1.05663342,
        -0.18782918,  0.99371959, -0.37143128,  1.34981227, -0.41599706,
        -0.13174454, -0.47155944,  0.09607662, -0.49266399, -0.20298469,
        -0.59096076,  1.7114372 ,  1.24395332, -0.24840774, -1.07262803]),
 array([0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.46153846, 0.        , 0.13333333, 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.25      ]))